In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import plotly.express as px

# Import your CRUD module
from aac_glue import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Read all records
df = pd.DataFrame.from_records(db.read({}))
if not df.empty and '_id' in df.columns:
    df['_id'] = df['_id'].astype(str)
    df.drop(columns=['_id'], inplace=True)
df = df.astype(str)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load and encode the logo and rabbit image
image_filename = 'GSL.png'
rabbit_filename = 'rabbit.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
encoded_rabbit = base64.b64encode(open(rabbit_filename, 'rb').read())

app.layout = html.Div([
    html.Center(
    html.A(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'width': '200px',
                'border': '2px solid #ddd',
                'border-radius': '8px',
                'box-shadow': '2px 2px 8px rgba(0, 0, 0, 0.1)'
            }
        ),
        href='https://www.snhu.edu'
    )
),
    html.Center(html.B(html.H1('CS-340 Grazioso Salvare Dashboard'))),
    html.Center(html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_rabbit.decode()), style={'width': '40px', 'margin-right': '10px'}),
        html.H3('Paper Rabbit Labs', style={'color': '#66CCFF', 'display': 'inline-block'})
    ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center'})),
    html.Hr(),

    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),

    html.Hr(),
#Renders table using pandas dataframes
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": d, "id": d, "deletable": False, "selectable": True} for d in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
    ),

    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={'border': '2px solid #ccc', 'padding': '10px', 'border-radius': '10px', 'margin-right': '10px'}),
        html.Div(id='map-id', className='col s12 m6', style={'border': '2px solid #000080', 'padding': '10px', 'border-radius': '10px'})
    ])
])

#############################################
# Callbacks
#############################################
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    else:
        query = {}

    results = db.read(query)
    frame = pd.DataFrame.from_records(results)
    if not frame.empty and '_id' in frame.columns:
        frame['_id'] = frame['_id'].astype(str)
        frame.drop(columns=['_id'], inplace=True)
    frame = frame.astype(str)
    return frame.to_dict('records')

@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
        breed_counts = dff['breed'].value_counts(normalize=True) * 100
        filtered_breeds = breed_counts[breed_counts >= 1.0].index.tolist()
        dff_filtered = dff[dff['breed'].isin(filtered_breeds)]

        fig = px.pie(dff_filtered, names='breed', title='Breed Distribution (>=1%)', hole=0.3)
        fig.update_traces(textinfo='percent+label')

        return [dcc.Graph(figure=fig)]
    return []

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    try:
        lat = float(dff.iloc[row, 13])
        lon = float(dff.iloc[row, 14])
        breed = dff.iloc[row, 4]
        name = dff.iloc[row, 9]
    except:
        return [html.P("Invalid data for map rendering.")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H1("Animal Name"), html.P(name)])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:10736/
